# Read Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
import xml.etree.ElementTree as ET
tree = ET.parse('training_data/nouns/access.n.xml')
root = tree.getroot()

In [3]:
documents = list(elem.text for elem in root.getchildren())

/home/misa/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if __name__ == '__main__':


In [4]:
from sklearn.cluster import KMeans

In [5]:
from nltk.corpus import wordnet

Number of meanings of "access" nouns from Wordnet:

In [6]:
def get_num_meanings(word, POS):
    length = 0
    for i in range(len(wordnet.synsets(word))):
        w = wordnet.synsets(word)[i]
        if w.pos() == POS and w.name().split('.')[0] == word:
            length += 1
    return length

Get definitions

In [7]:
def get_def(word, POS):
    definitions = []
    for w in wordnet.synsets(word):
        if w.name().split('.')[0] == word and w.pos() == POS:
            definitions.append(w.definition())
    return definitions

# Ex:
get_def('access', 'v')

# WORKS !!!

[u'obtain or retrieve from a storage device; as of information on a computer',
 u'reach or gain access to']

### Tfidf  vectorizer

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [9]:
# should try stemming later
lemmatizer = WordNetLemmatizer()

In [10]:
# strip punctuation
tokenizer = RegexpTokenizer(r'\w+')

In [11]:
# tokens = []
# for doc in documents:
#     words = tokenizer.tokenize(doc)
#     words_lem = [lemmatizer.lemmatize(word) for word in words]
#     tokens.append(words_lem)
    

In [12]:
# this will help to vectorize train and test documents so that they have the same dimensions. IF do fit_transform, will get error in dimens
vectorizer = TfidfVectorizer(stop_words='english').fit(documents)
vectors = vectorizer.transform(documents)

In [13]:
km = KMeans(n_clusters = get_num_meanings('access', 'n'))

In [14]:
km.fit(vectors)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [15]:
# km_labels is a numpy array that assign a cluster number to each example in documents
print len(km.labels_)

# Count how many elements in each cluster (order 0,1,2,3,4)
labels = np.bincount(km.labels_)
labels

11065


array([7201,  325, 1076,  950, 1513])

In [16]:
# sort by size of clusters, high to low
index =np.argsort(labels)[::-1]
index

array([0, 4, 2, 3, 1])

In [17]:
# Print definitions by order of clustering size
for i in index:
    print get_def('access','n')[i]

the right to obtain or make use of or take advantage of something (as services or membership)
the act of approaching or entering
a code (a series of characters or digits) that must be entered in some way (typed or dialed or spoken) to get the use of something (a telephone line or a computer or a local area network etc.)
(computer science) the operation of reading or writing stored information
a way of entering or leaving


## Predicting new document

In [21]:
from model import read_data, get_num_meanings, get_def, vectorize,\
                    kmean_fit, get_labels

In [23]:
test = 'pricing behavior could be used to determine when to remove incumbent LEC access services from price cap regulation'

In [50]:
test = 'this works great!'vector

In [51]:
[test]

['this works great!']

In [52]:
#v = vectorize([test])
v = vectorizer.transform([test])


In [53]:
v.shape

(1, 26405)

In [47]:
km.predict(v)

array([0], dtype=int32)

### Get definitions from labels

In [ ]:
# say we have a prediction of a number 